In [ ]:
!pip install openai==0.28 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.5

In [ ]:
import gradio as gr

In [ ]:
import openai

In [ ]:
# Initialize OpenAI client your openai key
openai.api_key = 'sk-None------'

In [ ]:
openai.api_key

'sk-None-9nHP0MjBrf3bui5toOYWT3BlbkFJe1I9h4UMGoZDjMIxYWf6'

In [ ]:
import requests

def get_current_weather(location, unit='celsius'):
    weather_api_key = "a7b109315b6ced6a9cd5177bdb98ca82"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [ ]:
print(get_current_weather("Ranchi"))

{'location': 'Ranchi', 'temperature': 26.06, 'weather': 'mist'}


In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [ ]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city, e.g. San Francisco'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [ ]:
#Function Definition and Initial Message Handling
def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot . Answer only in Celsius. If two cities are asked . then mention that you can only answer for one. if asked about Fahrenheit , politely tell we only have Celsius temperature . Answer the questions related to weather and temperature only. Else politely say no to the user"})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )

    #Handling Function Calls and Fetching Weather Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch weather data using the extracted arguments
        weather_data = get_current_weather(arguments['location'])

        # Append the function call and weather data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_current_weather", "content": str(weather_data)})

#magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide weather updates. Please ask me questions related to weather."



In [ ]:
#qc
weather_chat("كم درجة الحرارة في الشارقة؟ ")

'درجة الحرارة في الشارقة هي 37.16 درجة مئوية اليوم مع سماء صافية.'

In [ ]:
# Define Gradio interface
iface = gr.Interface(
    fn=weather_chat,
    inputs=gr.Textbox(label="Weather Queries"),
    outputs=gr.Textbox(label="Weather Updates"),
    title = "Weather Bot",
    description = "Ask me anything about weather!"
)

# Launch the Gradio interface
iface.launch(share="True")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://671a6bf86e1c4f8c85.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
